In [2]:
# import necessary packages
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from plotnine import *
import statsmodels.api as sm

from sklearn.linear_model import LinearRegression # Linear Regression Model
from sklearn.preprocessing import StandardScaler # Z-score variables
from sklearn.preprocessing import MinMaxScaler # Min-Max Normalization

from sklearn.model_selection import train_test_split # simple TT split cv

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

import os

In [16]:
# Iterate through each CSV file in the specified folder
data_folder = "../AllData/F_RotForwardX"

# Create an empty list to store the dataframes
dfs = []

# Iterate over the files in the folder
for filename in os.listdir(data_folder):
    if filename.endswith(".csv"):
        # Read the CSV file into a dataframe
        df = pd.read_csv(os.path.join(data_folder, filename))

        # Remove any data points that are not triggered by either the left or right trigger
        df.drop(df[(df['trigger_pull_amount_left'] == 0) & (df['trigger_pull_amount_right'] == 0)].index, inplace=True)
        
        # Extract the participant number from the file name (specified after "_subjID_")
        participant_num = int(filename.split("_subjID_")[1].split("_")[0])
        
        # Add the participant number as a column in the dataframe
        df.insert(0, 'participant_num', participant_num)
        
        # Append the dataframe to the list
        dfs.append(df)

# Concatenate all the dataframes into a single dataframe
all_F_rot_for_x_DF = pd.concat(dfs)

# Print the shape of the combined dataframe
print(all_F_rot_for_x_DF.shape)

(13304, 49)


In [17]:
# Group the dataframe by participant number
grouped_by_participant = all_F_rot_for_x_DF.groupby('participant_num')

# Calculate the number of participants
num_participants = len(grouped_by_participant)

# Calculate the number of rows and columns based on the number of participants (sqrt number of participants and add 1)
rows = cols = int(num_participants ** 0.5) + 1 

# Create a subplot grid figure
fig = make_subplots(rows=rows, cols=cols, subplot_titles=[f'Participant {participant_num}' for participant_num, _ in grouped_by_participant], specs=[[{'type': 'scatter3d'}]*cols]*rows)

row = 1  # Starting row index for subplot
col = 1  # Starting column index for subplot

# Iterate over the grouped data by participant
for i, (participant_num, group) in enumerate(grouped_by_participant, start=1):
    # Iterate over trials and add scatter traces
    for trial_num, trial_group in group.groupby('gesture_counter'):
        fig.add_trace(go.Scatter3d(
            x=trial_group['r_controller_translation_x'],
            y=trial_group['r_controller_translation_z'],
            z=trial_group['r_controller_translation_y'],
            mode='markers',
            marker=dict(
                size=2,
                color=trial_group.index,  # set color based on occurrence (use index as a unique identifier)
                colorscale='viridis',  # colorscale
                opacity=0.8,
            ),
            name=f'Trial {trial_num} - Right Controller',
            showlegend=False  # hide legend entry
        ), row=row, col=col)
        
        fig.add_trace(go.Scatter3d(
            x=trial_group['l_controller_translation_x'],
            y=trial_group['l_controller_translation_z'],
            z=trial_group['l_controller_translation_y'],
            mode='markers',
            marker=dict(
                size=2,
                color=trial_group.index,  # set color based on occurrence (use index as a unique identifier)
                colorscale='sunset_r',  # colorscale (reverse)
                opacity=0.8,
            ),
            name=f'Trial {trial_num} - Left Controller',
            showlegend=False  # hide legend entry
        ), row=row, col=col)
        
    # Update subplot indices for the next participant
    col += 1
    if col > cols:
        col = 1
        row += 1

# Set overall title for the figure
fig.update_layout(height=1000, width=1000, title_text='All Participants Rotate Forward (Freeform)') 
# fig.show()

# Specify output file path
output_path = "../Figures/Rot_For_X_Sub_1_to_26_Grid.html"

# Save figure as an HTML file
pio.write_html(fig, file=output_path, auto_open=True)

In [19]:
# Iterate through each CSV file in the specified folder
data_folder = "../AllData/F_RotForwardX"

# Create an empty list to store the dataframes
dfs = []

# Create a dictionary to track the number of sessions for each participant
participant_sessions = {}

# Iterate over the files in the folder
for filename in os.listdir(data_folder):
    if filename.endswith(".csv"):
        # Read the CSV file into a dataframe
        df = pd.read_csv(os.path.join(data_folder, filename))

        # Remove any data points that are not triggered by either the left or right trigger
        df.drop(df[(df['trigger_pull_amount_left'] == 0) & (df['trigger_pull_amount_right'] == 0)].index, inplace=True)

        # Extract the participant number from the file name (specified after "_subjID_")
        participant_num = int(filename.split("_subjID_")[1].split("_")[0])
        
        # Check if participant already has sessions
        if participant_num in participant_sessions:
            session_num = participant_sessions[participant_num] + 1
            participant_sessions[participant_num] = session_num
        else:
            session_num = 1
            participant_sessions[participant_num] = session_num
        
        # Add the participant number as a column in the dataframe
        df.insert(0, 'participant_num', participant_num)

        # Create a unique identifier for the session
        session_identifier = f"{session_num}"
        
        # Add the session identifier as a column in the dataframe
        df.insert(0, 'session_identifier', session_identifier)
        
        # Append the dataframe to the list
        dfs.append(df)

# Concatenate all the dataframes into a single dataframe
all_F_rot_for_x_DF = pd.concat(dfs)

# Print the shape of the combined dataframe
print(all_F_rot_for_x_DF.shape)

In [29]:
# Group the dataframe by participant number and session identifier
grouped_by_participant_session = all_F_rot_for_x_DF.groupby(['participant_num', 'session_identifier'])

# Calculate the number of participants and session
num_participants = len(grouped_by_participant_session)

# Calculate the number of rows and columns based on the number of participants and sessions
rows = cols = int(num_participants ** 0.5) + 1

# Create a subplot grid figure
fig = make_subplots(rows=rows, cols=cols, subplot_titles=[f'P {participant_num}, S {session_id}' for (participant_num, session_id), _ in grouped_by_participant_session], specs=[[{'type': 'scatter3d'}]*cols]*rows)

row = 1  # Starting row index for subplot
col = 1  # Starting column index for subplot

# Iterate over the grouped data by participant and session
for i, ((participant_num, session_id), group) in enumerate(grouped_by_participant_session, start=1):
    # Iterate over trials for each participant and session, and add scatter traces
    for trial_num, trial_group in group.groupby('gesture_counter'):
        # Add scatter trace for right controller
        fig.add_trace(go.Scatter3d(
            x=trial_group['r_controller_translation_x'],
            y=trial_group['r_controller_translation_z'],
            z=trial_group['r_controller_translation_y'],
            mode='markers',
            marker=dict(
                size=2,
                color=trial_group.index,  # set color based on occurrence (use index as a unique identifier)
                colorscale='viridis',  # colorscale
                opacity=0.8,
            ),
            name=f'Trial {trial_num} - Right Controller',
            showlegend=False  # hide legend entry
        ), row=row, col=col)
        
        # Add scatter trace for left controller
        fig.add_trace(go.Scatter3d(
            x=trial_group['l_controller_translation_x'],
            y=trial_group['l_controller_translation_z'],
            z=trial_group['l_controller_translation_y'],
            mode='markers',
            marker=dict(
                size=2,
                color=trial_group.index,  # set color based on occurrence (use index as a unique identifier)
                colorscale='sunset_r',  # colorscale (reverse)
                opacity=0.8,
            ),
            name=f'Trial {trial_num} - Left Controller',
            showlegend=False  # hide legend entry
        ), row=row, col=col)
        
    # Update subplot indices for the next participant and session
    col += 1
    if col > cols:
        col = 1
        row += 1

# Set overall title for the figure
fig.update_layout(height=1000, width=1000, title_text='All Participants By Session Rotate Forward (Freeform)')  # Set the overall title for the figure
# fig.show()

# Specify the output file path
output_path = "C:\\Users\\vrelax\\Desktop\\VRelax\\gestureInterface\\Figures\\Rot_For_X_Sub_1_to_26_Session_37_Grid.html"

# Save the figure as an HTML file
pio.write_html(fig, file=output_path, auto_open=True)

ValueError: 
The 'specs' argument to make_subplots must be a 2D list of dictionaries with dimensions (1 x 1).
    Received value of type <class 'list'>: [[{'type': 'scatter3d'}, {'type': 'scatter3d'}, {'type': 'scatter3d'}, {'type': 'scatter3d'}], [{'type': 'scatter3d'}, {'type': 'scatter3d'}, {'type': 'scatter3d'}, {'type': 'scatter3d'}], [{'type': 'scatter3d'}, {'type': 'scatter3d'}, {'type': 'scatter3d'}, {'type': 'scatter3d'}], [{'type': 'scatter3d'}, {'type': 'scatter3d'}, {'type': 'scatter3d'}, {'type': 'scatter3d'}]]

In [30]:
cleaned_data_folder_path = "C:\\Users\\vrelax\\Desktop\\VRelax\\gestureInterface\\CleanedData"
figures_folder_path = "C:\\Users\\vrelax\\Desktop\\VRelax\\gestureInterface\\Figures"

figure_gesture_dict = {}

# Iterates through CleanedData directory to make subfolders in Figures directory and make a dictionary
# of data where the 13 keys are gesture types and its values are two lists representing freeform and instructional.
# Each list will contain the path to the 37 csv data files.
for root, sub_folders, files in os.walk(cleaned_data_folder_path):
    for file in files:
        # Splits file name for gesture and session type identification
        file_split = file.split("_")
        file_gesture = file_split[3]
        file_session_type = file_split[2]

        # Ignores the thank you files
        if (file_session_type != 'F') and (file_session_type != 'I'):
            continue
        
        # Make gesture folder in Figures directory
        gesture_in_figures_path = os.path.join(figures_folder_path, file_gesture)
        if not os.path.exists(gesture_in_figures_path):
            os.makedirs(os.path.join(gesture_in_figures_path, "Freeform"))
            os.makedirs(os.path.join(gesture_in_figures_path, "Instructional"))

        # If a gesture is not in the dictionary, make the gesture a new key with list values freeform and instructional
        if file_gesture not in figure_gesture_dict:
            freeform_folder = []
            instructional_folder = []
            figure_gesture_dict[file_gesture] = [freeform_folder, instructional_folder]

        if file_session_type == 'F':
            figure_gesture_dict[file_gesture][0].append(os.path.join(root, file))
        else:
            figure_gesture_dict[file_gesture][1].append(os.path.join(root, file))
        


# Iterates through each gesture in the dictionary to create plots
for gesture in figure_gesture_dict:
    for session_type in figure_gesture_dict[gesture]:
        # Create an empty list to store the dataframes
        dfs = []
        # Create a dictionary to track the number of sessions for each participant
        participant_sessions = {}

        for file_num in range(len(session_type)):
            # Read the CSV file into a dataframe
            df = pd.read_csv(session_type[file_num])
            filename = os.path.basename(session_type[file_num])

            # Remove any data points that are not triggered by either the left or right trigger
            df.drop(df[(df['trigger_pull_amount_left'] == 0) & (df['trigger_pull_amount_right'] == 0)].index, inplace=True)

            # Extract the participant number from the file name (specified after "_subjID_")
            participant_num = int(filename.split("_subjID_")[1].split("_")[0])
        
            # Check if participant already has sessions
            if participant_num in participant_sessions:
                session_num = participant_sessions[participant_num] + 1
                participant_sessions[participant_num] = session_num
            else:
                session_num = 1
                participant_sessions[participant_num] = session_num
        
            # Add the participant number as a column in the dataframe
            df.insert(0, 'participant_num', participant_num)

            # Create a unique identifier for the session
            session_identifier = f"{session_num}"
        
            # Add the session identifier as a column in the dataframe
            df.insert(0, 'session_identifier', session_identifier)
        
            # Append the dataframe to the list
            dfs.append(df)

        # Concatenate all the dataframes into a single dataframe
        all_data_for_gesture_DF = pd.concat(dfs)




        # Group the dataframe by participant number and session identifier
        grouped_by_participant_session = all_data_for_gesture_DF.groupby(['participant_num', 'session_identifier'])

        # Calculate the number of participants and session
        num_participants = len(grouped_by_participant_session)

        # Calculate the number of rows and columns based on the number of participants and sessions
        rows = cols = int(num_participants ** 0.5) + 1

        # Create a subplot grid figure
        fig = make_subplots(rows=rows, cols=cols, subplot_titles=[f'P {participant_num}, S {session_id}' for (participant_num, session_id), _ in grouped_by_participant_session], specs=[[{'type': 'scatter3d'}]*cols]*rows)

        row = 1  # Starting row index for subplot
        col = 1  # Starting column index for subplot


        # Iterate over the grouped data by participant and session
        for i, ((participant_num, session_id), group) in enumerate(grouped_by_participant_session, start=1):
            # Iterate over trials for each participant and session, and add scatter traces
            for trial_num, trial_group in group.groupby('gesture_counter'):
                # Add scatter trace for right controller
                fig.add_trace(go.Scatter3d(
                    x=trial_group['r_controller_translation_x'],
                    y=trial_group['r_controller_translation_z'],
                    z=trial_group['r_controller_translation_y'],
                    mode='markers',
                    marker=dict(
                        size=2,
                        color=trial_group.index,  # set color based on occurrence (use index as a unique identifier)
                        colorscale='viridis',  # colorscale
                        opacity=0.8,
                    ),
                    name=f'Trial {trial_num} - Right Controller',
                    showlegend=False  # hide legend entry
                ), row=row, col=col)
        
                # Add scatter trace for left controller
                fig.add_trace(go.Scatter3d(
                    x=trial_group['l_controller_translation_x'],
                    y=trial_group['l_controller_translation_z'],
                    z=trial_group['l_controller_translation_y'],
                    mode='markers',
                    marker=dict(
                        size=2,
                        color=trial_group.index,  # set color based on occurrence (use index as a unique identifier)
                        colorscale='sunset_r',  # colorscale (reverse)
                        opacity=0.8,
                    ),
                    name=f'Trial {trial_num} - Left Controller',
                    showlegend=False  # hide legend entry
                ), row=row, col=col)

            # Update subplot indices for the next participant and session
            col += 1
            if col > cols:
                col = 1
                row += 1

        # Set overall title and output path for the figure
        gesture_name = ""
        session_name = ""
        html_file_name = ""
        html_output_path = figures_folder_path + "\\"
        figures_title = "All Participants By Session "
        
        # Splits the first file in the folder by '_' to get the session and gesture type
        file = os.path.basename(session_type[0])
        session_split = file.split("_")[2]
        gesture_split = file.split("_")[3]

        html_output_path += gesture_split + "\\"

        # Gets the session type and adds to the html output path
        if session_split == 'F':
            session_name = "Freeform"
            html_output_path += "Freeform\\"
        else:
            session_name = "Instructional"
            html_output_path += "Instructional\\"

        # Iterates through the gesture name. Separates the words with a space for the title and _ for the html output path
        for i in range(len(gesture)):
            if ((gesture_split[i].isupper() == True) and (i != 0)):
                gesture_name += " " + gesture[i]
                html_file_name += "_" + gesture[i]
            else:
                gesture_name += gesture[i]   
                html_file_name += gesture[i]
        
        html_output_path += html_file_name + "_Sub_1_to_26_Session_37_Grid.html"
        figures_title += gesture_name + " (" + session_name + ")"

        fig.update_layout(height=1000, width=1000, title_text=figures_title)  # Set the overall title for the figure
        # fig.show()

        # Specify the output file path
        print(html_output_path)
        print(figures_title)
        # Save the figure as an HTML file
        if not os.path.exists(html_output_path):
            pio.write_html(fig, file=html_output_path, auto_open=True)



C:\Users\vrelax\Desktop\VRelax\gestureInterface\Figures\BoxSelect\Freeform\Box_Select_Sub_1_to_26_Session_37_Grid.html
All Participants By Session Box Select (Freeform)
C:\Users\vrelax\Desktop\VRelax\gestureInterface\Figures\BoxSelect\Instructional\Box_Select_Sub_1_to_26_Session_37_Grid.html
All Participants By Session Box Select (Instructional)
C:\Users\vrelax\Desktop\VRelax\gestureInterface\Figures\PanDown\Freeform\Pan_Down_Sub_1_to_26_Session_37_Grid.html
All Participants By Session Pan Down (Freeform)
C:\Users\vrelax\Desktop\VRelax\gestureInterface\Figures\PanDown\Instructional\Pan_Down_Sub_1_to_26_Session_37_Grid.html
All Participants By Session Pan Down (Instructional)
C:\Users\vrelax\Desktop\VRelax\gestureInterface\Figures\PanLeft\Freeform\Pan_Left_Sub_1_to_26_Session_37_Grid.html
All Participants By Session Pan Left (Freeform)
C:\Users\vrelax\Desktop\VRelax\gestureInterface\Figures\PanLeft\Instructional\Pan_Left_Sub_1_to_26_Session_37_Grid.html
All Participants By Session Pan L